# Hypothesis and Inference
##### It is the mark of a truly intelligent person to be moved by statistics. -George Bernard Shaw

In [13]:
%%capture
# To supress the output when calling another file
%run ./Probability.ipynb

### Statistical hypothesis testing

In [2]:
def normal_approximation_to_binomial(n,p):
    """finds mu and sigma corresponding to a Binomial(n,p)"""
    mu    = n*p
    """variance = E(X^2)-E(X)^2 +(E(X)-E(X))
                = E(X[X-1])+E(X)-E(X)^2 
                = np*(p*(n-1))+np-(np)^2 
                = np(1-p) """
    sigma = math.sqrt(n*p*(1-p)) 
    
    return mu,sigma

Whenever a random variable follows Normal distribution, normal_cdf can be used to figure out  the probability that its realized value lies within (or outside) a particular interval

In [3]:
# The normal_cdf is the probability the variable is below a threshold

normal_probability_below = normal_cdf

# it's above a threshold if it's not below the threshold
def normal_probability_above(lo,mu=0,sigma=1):
    return 1-normal_cdf(lo,mu,sigma)

# it's between if it's less than hi, but not less than lo
def normal_probability_between(lo,hi,mu=0,sigma=1):
    return normal_probability_below(hi,mu,sigma) - normal_probability_below(lo,mu,sigma)

# it's outside if it's not in between
def normal_probability_outside(lo,hi,mu=0,sigma=1):
    return 1-normal_probability_between(lo,hi,mu,sigma)

def normal_upper_bound(probability, mu=0, sigma=1):
    """returns z for which P(Z<=z) = probability"""
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu=0, sigma=1):
    """returns z for which P(Z>=z) = probability"""
    return inverse_normal_cdf((1-probability),mu,sigma)

def normal_two_sided_bound(probability, mu=0, sigma=1):
    """returns the symmetric (about mean) bounds"""
    lo_probability = (1-probability)/2
    hi_probability = probability+lo_probability
    lower_bound    = normal_upper_bound(lo_probability,mu,sigma)
    upper_bound    = normal_upper_bound(hi_probability,mu,sigma)
    return lower_bound,upper_bound

### Example flipping a coin
We want to test weather a coin is fair. So we'll make the assumption abpout probability p of coin landing Heads (0.5 for our case). So, our null hypothesis is that coin is fair- i.e. p = 0.5. We'll test this against the alternative hypothesis p != 5.
In particular our test will involve flipping the coin *n* number of times and counting number of Heads X. Since each coin flip is a Bernoulli trial, which means X should be a Binomial(n,p) random variable. Which can be approximated as normal. In Particular we choose to flip the coin 1000 times. 
##### If our hypothesis of fairness is TRUE, X should be distributed approximately normally with mean 500 and standard deviation 15.8. 
Also, we need to make a decision about *significance* - how willing we are to make *type 1 error* ("False positive"), in which we reject *H0* even though it's true (let's choose 5%).
Consider a test, where we reject *H0* if it falls outside the bounds given by 95% probability bounds. 
Assuming *p* really equals 0.5 (i.e. H0 is TRUE), there is just a 5% chance we observe an X that lies outside this interval, which is the exact significance we wanted.

In [4]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000,0.5)
print("mean:",mu_0, "std:",sigma_0)
print("95% probability bounds:", normal_two_sided_bound(0.95, mu_0, sigma_0))

mean: 500.0 std: 15.811388300841896
95% probability bounds: (469.01026640487555, 530.9897335951244)


### Power of a test (likelihood of rejecting the null hypothesis when it's false or 1 - Type 2 error)
<https://www.moresteam.com/whitepapers/download/power-stat-test.pdf>

|     *    |   H0 is TRUE      |  H0 is FALSE                    |
|:---------|:-----------------:|:-------------------------------:|
|Accepted  | 95%               | Type 2 error                    |
|Rejected  | Type 1 error (5%) | Power of test 1-P(Type 2 error) |


To measure the probability of making a type 2 error, we have to specify what exactly H0 being false means
(It is generally a specific condition under the alternative hypothesis)
For this analysis assume that p = 0.55 (or the coin is slightly biased towards Heads) and check what happens.

In [5]:
# 95% bounds based on assumption p is 0.5
lo, hi = normal_two_sided_bound(0.95, mu_0, sigma_0)

# actual mu and sigma based on p=0.55
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

# a type 2 error means we fail to reject null hypothesis even when it's false.
# which will happen when if in our original Test, X is still in our 95% confidence interval
type_2_probability = normal_probability_between(lo,hi,mu_1,sigma_1)
power = 1 - type_2_probability
print("Power:", round(power,2))

Power: 0.89


### One tailed test
If instead of our original Hypothesis coin is fair (p=0.5) we are interested in coin is not biased towards Head or that p<=0.5. In that case we want a *one sided test* that rejects null hypothesis when *X* is much large than 500 but not when it is less than 500. Suppose we still want significance of 5% i.e. error of rejecting null hypothesis when it's TRUE. This means we have to find the cutoff below which 95% probability lies.

In [6]:
hi = normal_upper_bound(0.95,mu_0,sigma_0)
print("We will reject null hypothesis if observed value is above:", hi)

type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability
print("Power:", round(power,2))

We will reject null hypothesis if observed value is above: 526.0073585242053
Power: 0.94


This is a more powerful test than before since it no longer rejects *H0* if it's below lower bound (469) but only when it's above an upper bound (526)

### p-values
An alternate way to think about the tests above is that instead of choosing bounds based on some probability cutoff, we compute the probability assuming *H0* is TRUE that we would see a value at least as extreme as the one actually observed.

In [7]:
# For our two-sided test of weather coin is fair, we compute:

def two_sided_p_value(x, mu=0, sigma=1):
    if x>=mu:
        return 2*normal_probability_above(x,mu,sigma)
    else:
        return 2*normal_probability_below(x,mu,sigma)
    
# So if we were to see 530 Heads, we would compute: (taking into account continuity correction)
p_value_530_heads = two_sided_p_value(529.5,mu_0,sigma_0)
print("p-value for observing 530 Heads:", round(p_value_530_heads,3))

# Another way to estimate the p-value in this scenario is by simulation
extreme_value_count = 0
num_of_experiments   = 10000
num_coin_tosses_per_experiment = 1000
for _ in range(num_of_experiments):
    num_heads = sum([1 if random.random()>0.5 else 0                  # count # of Heads 
                     for _ in range(num_coin_tosses_per_experiment)]) # in 1000 flips
    if num_heads >= 530 or num_heads <= 470:                          # and count how often the # is 'extreme'
        extreme_value_count += 1

p_value_simulation = extreme_value_count/num_of_experiments
print("p-value from simulation:", round(p_value_simulation,3))

p-value for observing 530 Heads: 0.062
p-value from simulation: 0.061


Since the p value > 5% significance level so it is not rare to observe 530 Heads and we do not reject *H0*.
If we instead observed 532 Heads, the *p-value* would be:

In [8]:
p_value_532_heads = two_sided_p_value(531.5, mu_0, sigma_0)
print("p-value for observing 532 Heads:", round(p_value_532_heads,3))

p-value for observing 532 Heads: 0.046


Which is smaller than the 5% significane level, so it is rare to observe such a value and if observed then *H0* should be rejected.

### Confidence intervals
We've been interested in the value of heads probability p, which is a parameter of the unknown Heads distribution. We can use a third approach to construct a confidence interval around the observed value of parameter (p in our case).
For example if 525 Heads are observed in a sample of 1000 flips, then we estimate p equals 0.525.
How confident can we be about this estimate? From Central limit theorem we know that since these are Bernoulli variables and they should be approximately Normally distributed with mean p and standard deviation sqrt(np(1-p)/1000).


In [9]:
# For this case 
p_hat = 525/1000
mu    = p_hat
sigma = math.sqrt(p_hat*(1-p_hat)/1000)
(lo, hi) = normal_two_sided_bound(0.95, mu, sigma)
print("95% confidence bounds around observed probability:", (round(lo,3), round(hi,3)))

95% confidence bounds around observed probability: (0.494, 0.556)


Since *0.5* falls under the 95% confidence bounds so *H0* is not rejected

### P-hacking
If we are setting out to find "significant" results, we usually can. Test enough hypothesis against data set and one of them will most certainly appear significant. Remove right outliers, and the results can appear significant.
This is sometimes called p-hacking and is in someway a consequence of "inference from p-value framework".
https://www.quora.com/What-is-p-hacking


In [10]:
def run_experiment():
    """flip a fair coin 1000 times, True = Heads, False = tails"""
    return [random.random() > 0.5 for _ in range(1000)]

def reject_fairness(experiment):
    """using 5% significance levels and outcomes of an experiment 
        determine weather the experiment indicates if the coin is biased (not fair)"""
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [run_experiment() for _ in range(1000)]

# Count number of experiments where fairness hypothesis os rejected with 5% significance
num_rejections = len([experiment 
                     for experiment in experiments if reject_fairness(experiment)])

print(num_rejections)

46


This can falsefully lead to a conclusion that the coin is biased, or in other words the results are significant. Because given enough number of experiments there will always be some results that are significant. P-hacking means using this fact to distort findings.

### Running A/B test (two sample t-test really)
Let's say we want to compare which of the 2 Ads is effective. A being shown to *Na* number of peope and *na* people click on it and B being shown to *Nb* number of people and *nb* people click on it. We need to determine if there is a significant difference between these Ads. We can treat each Ad view as a Bernoulli trial. And if *Na* is large then *na/Na* should be approximately normally distributed with mean *pa* and standard deviation *sqrt(pa(1-pa)/Na*. Same is TRUE for B.

In [11]:
def estimated_parameters(N,n):
    p = n/N
    sigma = math.sqrt(p*(1-p)/N)
    return p,sigma

def a_b_statistic(N_A, n_A, N_B, n_B):
    p_A, sigma_A = estimated_parameters(N_A,n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B-p_A)/(math.sqrt(sigma_A**2+sigma_B**2))

If we assume the two Normals are independent, then their difference should also be normal with mean pb - pa and standard deviation sqrt(sigma_a^2 + sigma_b^2).
##### This may not be entirely correct since we don't know the population standard deviation and are trying to infer population standard deviation using samples, which means the a_b_statistic is essentially a t-distribution. But for large enough data sets it's close enough.
Let's assume A gets 200/1000 clicks and B gets 180/1000 clicks.

In [15]:
t = a_b_statistic(1000,200,1000,180)
print("t-statistic for B and A:", round(t,2))
p_value = round(two_sided_p_value(t),2)
# Does not need mu and sigma as it is already a Z value

print("The probability of seeing such a large difference if means were actually equal would be:",p_value) 
print("So null hypothesis that means are equal is not rejected")

t-statistic for B and A: -1.14
The probability of seeing such a large difference if means were actually equal would be: 0.25
So null hypothesis that means are equal is not rejected
